# This notebook is used to run the python code through Jupyter as if it was a command line. So we run python files, and can use commands such as mpirun to run stuff in parallel.
# Need to put a '!' in fron of every command meant for the command line

In [1]:
#Increasing the width of the notebook (visual difference only)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#Very nice link tutorial for MPI with Python:
#https://rabernat.github.io/research_computing/parallel-programming-with-mpi-for-python.html

#IMPORTANT: It turn out that the error coming from using too much memory doesnt seem to happen when using
#MPI! This error: 'PETSc error code is: 76' This means we can use bigger, better meshes and not be
#limited by memory if we run in parallel

#Converting the notebook to a runnable python file 
!jupyter nbconvert --to script MONDtest_NoFunctions_MPI.ipynb

[NbConvertApp] Converting notebook MONDtest_NoFunctions_MPI.ipynb to script
[NbConvertApp] Writing 48713 bytes to MONDtest_NoFunctions_MPI.py


In [2]:
#Running the full script in parallel using MPI. Optimal number of processes:8. This includes two threads
#per core.
! mpirun -n 8 python3 MONDtest_NoFunctions_MPI.py

Process 0: Generating mesh with CGAL 3D mesh generator
Process 0: Computed global bounding box tree with 15 boxes.Process 1: Computed global bounding box tree with 15 boxes.
Process 2: Computed global bounding box tree with 15 boxes.
Process 3: Computed global bounding box tree with 15 boxes.
Process 4: Computed global bounding box tree with 15 boxes.
Process 5: Computed global bounding box tree with 15 boxes.
Process 6: Computed global bounding box tree with 15 boxes.
Process 7: Computed global bounding box tree with 15 boxes.

Process 0: No Jacobian form specified for nonlinear variational problem.
Process 0: Differentiating residual form F to obtain Jacobian J = F'.
Process 1: No Jacobian form specified for nonlinear variational problem.
Process 1: Differentiating residual form F to obtain Jacobian J = F'.
Process 2: No Jacobian form specified for nonlinear variational problem.
Process 2: Differentiating residual form F to obtain Jacobian J = F'.
Process 4: No Jacobian form specifie